In [1]:
from prj.config import DATA_DIR
from prj.data.data_loader import DataConfig, DataLoader
import polars as pl

data_args = {}
config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)

2024-12-29 01:39:55.061317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-29 01:39:55.061358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-29 01:39:55.062745: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-29 01:39:55.070076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 01:39:55.870294: W tensorflow/compiler/tf2

In [2]:
start_dt, end_dt = 1100, 1110
start_val_dt = 1108

In [3]:
complete_ds = loader.load(start_dt, end_dt)

val_ds = complete_ds.filter(pl.col('date_id').ge(start_val_dt))
val_ds.collect().head(5)

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
1108,0,0,1.599182,1.25368,-0.437875,1.548298,1.877996,1.987537,-1.361107,0.715413,-0.534521,0.302156,11,7,76,-0.671134,0.919318,-0.23278,null,0.572328,null,-1.704065,-0.915655,0.825131,-0.228274,0.56992,1.112303,-0.933503,-0.998992,1.139633,1.222561,0.857662,0.047741,0.517775,-0.173189,null,…,null,null,-1.470734,null,-0.826701,0.366417,null,0.211672,-0.013402,-0.711842,0.02793,0.383899,0.319894,null,null,-0.529676,0.347533,-0.305567,-1.174261,2.219099,-0.190434,null,null,-0.239844,-0.167164,-0.151576,-0.35227,-0.664202,-0.006456,0.213953,-0.144088,-0.809349,-0.137964,0.297297,-1.567391,-0.518465,6
1108,0,1,2.844544,2.107626,0.634461,0.900853,2.303407,2.171719,-0.871822,0.433323,-0.304671,0.240534,11,7,76,-1.075586,1.673318,-0.319468,null,-0.056212,null,-1.617553,-0.928436,0.130291,0.006819,2.349689,1.724608,-0.875158,-0.176288,0.51666,1.788792,1.231096,-0.34207,-0.123342,0.006035,null,…,null,null,-1.178793,null,-1.270172,1.390535,null,1.001953,0.182568,-0.711842,-0.321874,-0.164353,-0.228116,-1.584387,-1.48613,-0.856901,0.728709,-0.29794,-0.725641,2.133685,-0.09719,null,null,-0.20097,-0.245658,-0.328485,-0.292093,-0.291818,-0.210165,-0.199149,-0.598838,-1.154606,0.206256,-0.519193,-1.083733,0.715887,6
1108,0,2,1.778899,1.098838,0.408612,1.382891,1.297657,2.518946,-0.963115,0.691736,-0.334735,0.253715,81,2,59,-0.854383,-0.100864,-0.812462,null,-0.172732,null,-2.131718,-1.850823,-0.939381,-0.251988,0.590109,0.110248,-1.154203,-0.847603,0.848478,1.35906,1.375511,-0.429401,-0.455752,-0.250771,null,…,null,null,-2.786971,null,-1.279411,-0.033275,null,-0.852217,-0.420891,-0.711842,-0.178966,-0.10043,-0.106295,-2.168841,-1.996272,-0.932895,-0.09625,-0.554614,-1.194544,-0.077133,-0.452011,null,null,4.90193,8.509361,13.77626,16.06304,-0.121137,-0.122312,-0.135348,-1.413297,-0.520816,0.370638,-2.777546,-0.391677,0.677883,6
1108,0,3,1.230088,1.073081,0.116436,1.593973,1.207443,2.219879,-0.576148,0.509578,-0.212684,0.121532,4,3,11,-0.330236,7.445848,1.327974,null,-0.188279,null,-1.233456,-2.028468,-0.194337,0.049778,0.009243,-0.19984,0.888055,1.039983,-0.3943,-0.463027,-0.953243,-0.665958,-0.658918,0.047487,null,…,null,null,-1.236717,null,-0.556,1.513677,null,3.488591,1.132235,-0.711842,-0.252953,-0.270835,-0.271094,-2.050989,-1.501235,-0.628259,1.939634,0.022731,-0.300654,8.424384,1.322707,null,null,0.499284,0.500686,-0.056693,-0.037714,0.666928,0.058977,0.312081,-2.393661,-0.226595,-1.017476,-3.332203,-0.305786,-1.549229,6
1108,0,4,1.57557,1.393097,0.683312,1.041991,1.166212,2.003079,-0.410651,0.394495,-0.23376,0.115619,15,1,9,-0.722064,-0.091613,-0.638488,null,-0.020833,null,-1.600799,-1.023754,1.333991,0.038441,-0.04504,-0.510163,0.520042,1.171758,-1.428435,-1.115971,-0.219436,-0.552869,-0.737578,0.046018,null,…,null,null,-1.363041,null,-1.086848,1.488567,null,-1.27407,-0.939332,-0.711842,-0.226862,0.043165,-0.108489,-1.151456,-2.267298,-0.616098

In [5]:
import os
from typing import List, Optional, Tuple, Dict, Any

import pandas as pd
import polars as pl
import numpy as np

import torch
import torch.nn as nn
from torch import Tensor
import pytorch_lightning as L
from torch.utils.data import Dataset, DataLoader
import time

import kaggle_evaluation.jane_street_inference_server

In [6]:
def weighted_r2_score(preds, targets, weights):
    ss_res = (weights * (targets - preds) ** 2).sum()
    ss_tot = (weights * (targets ** 2)).sum()
    return 1 - ss_res / ss_tot if ss_tot > 0 else 0.0

In [7]:
class WeightedMSELoss(nn.Module):
    def __init__(self):
        super(WeightedMSELoss, self).__init__()
    
    def forward(self, predictions: Tensor, targets: Tensor, weights: Tensor) -> Tensor:
        squared_diff = (predictions - targets) ** 2
        weighted_squared_diff = weights * squared_diff
        return weighted_squared_diff.sum() / weights.sum()

In [8]:
class SimpleNNModel(nn.Module):
    def __init__(self, input_features, hidden_dims=[], dropout_rate=0.1, output_dim=1, 
                 use_tanh=False, final_mult=1.0):
        super(SimpleNNModel, self).__init__()
        self.final_mult = final_mult
        self.use_tanh = use_tanh
        
        layers = [nn.BatchNorm1d(input_features), nn.Dropout(dropout_rate)]
        in_features = input_features

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(in_features, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.SiLU())
            layers.append(nn.Dropout(dropout_rate))
            in_features = hidden_dim
            
        layers.append(nn.Linear(in_features, output_dim))
        if self.use_tanh:
            layers.append(nn.Tanh())
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.final_mult * self.model(x)

In [9]:
class JaneStreetBaseModel(L.LightningModule):

    def __init__(self, 
                 model: nn.Module,
                 losses: List[nn.Module] | nn.Module, 
                 loss_weights: List[float], 
                 l1_lambda: float = 1e-4,
                 l2_lambda: float = 1e-4,
                 optimizer: str = 'Adam',
                 optimizer_cfg: Dict[str, Any] = dict(),
                 scheduler: str = None,
                 scheduler_cfg: Dict[str, Any] = dict()):
        super(JaneStreetBaseModel, self).__init__()   
        assert isinstance(losses, nn.Module) or len(losses) == len(loss_weights), 'Each loss must have a weight'
        assert len(loss_weights) == 0 or min(loss_weights) > 0, 'Losses must have positive weights'
        self.model = model
        losses = [losses] if isinstance(losses, nn.Module) else losses
        self.losses = nn.ModuleList(losses) 
        self.loss_weights = [1.0] if isinstance(losses, nn.Module) else loss_weights
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda
        self.optimizer_name = optimizer
        self.optimizer_cfg = optimizer_cfg
        self.scheduler_name = scheduler
        self.scheduler_cfg = scheduler_cfg
        self.acc_metrics = dict(ss_res=0.0, ss_tot=0.0, abs_err_sum=0.0, weights_sum=0.0)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y, weights = batch
        y_hat = self.forward(x).squeeze()
        loss = self._compute_loss(y_hat, y, weights)
        reg_loss, l1_loss, l2_loss = self._regularization_loss()
        metrics = dict()
        with torch.no_grad():
            metrics['train_wmse'] = (weights * (y_hat - y) ** 2).sum() / weights.sum()
            metrics['train_wmae'] = (weights * (y_hat - y).abs()).sum() / weights.sum()
            metrics['train_wr2'] = weighted_r2_score(y_hat, y, weights)
        metrics['train_loss'] = loss
        if self.l1_lambda > 0:
            metrics['train_l1_reg'] = l1_loss
        if self.l2_lambda > 0:
            metrics['train_l2_reg'] = l2_loss
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=x.size(0))
        return loss + reg_loss

    def validation_step(self, batch, batch_idx):
        x, y, weights = batch
        y_hat = self.forward(x).squeeze()
        loss = self._compute_loss(y_hat, y, weights)
        ss_res_step = (weights * (y_hat - y) ** 2).sum()
        ss_tot_step = (weights * (y ** 2)).sum()
        abs_err_step = (weights * (y_hat - y).abs()).sum()
        weights_sum_step = weights.sum()
        self.acc_metrics['ss_res'] += ss_res_step
        self.acc_metrics['ss_tot'] += ss_tot_step
        self.acc_metrics['abs_err_sum'] += abs_err_step
        self.acc_metrics['weights_sum'] += weights_sum_step
        self.log('val_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))

    def on_validation_epoch_end(self):
        metrics = dict()
        metrics['val_wmse'] = self.acc_metrics['ss_res'] / self.acc_metrics['weights_sum']
        metrics['val_wmae'] = self.acc_metrics['abs_err_sum'] / self.acc_metrics['weights_sum']
        metrics['val_wr2'] = 1 - self.acc_metrics['ss_res'] / self.acc_metrics['ss_tot']
        self.acc_metrics = dict(ss_res=0.0, ss_tot=0.0, abs_err_sum=0.0, weights_sum=0.0)
        self.log_dict(metrics, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    def _compute_loss(self, preds, targets, weights):
        loss = 0
        for i in range(len(self.losses)):
            loss += self.losses[i](preds, targets, weights=weights) * self.loss_weights[i]
        return loss
    
    def _regularization_loss(self):
        reg_loss = 0
        l1_loss = 0
        l2_loss = 0
        
        if self.l1_lambda > 0:
            l1_loss = sum(p.abs().sum() for p in self.parameters())
            reg_loss += l1_loss * self.l1_lambda
            
        if self.l2_lambda > 0:
            l2_loss = sum(p.pow(2).sum() for p in self.parameters())
            reg_loss += l2_loss * self.l2_lambda
            
        return reg_loss, l1_loss, l2_loss

    def configure_optimizers(self):
        optimizer = getattr(torch.optim, self.optimizer_name)(self.parameters(), **self.optimizer_cfg)
        if self.scheduler_name is None:
            return optimizer
        scheduler = getattr(torch.optim.lr_scheduler, self.scheduler_name)(optimizer, **self.scheduler_cfg)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_wr2',
            }
        }

In [10]:
training_base_model = SimpleNNModel(79, hidden_dims=[256], dropout_rate=0.3, final_mult=5.0, use_tanh=True)
training_model = JaneStreetBaseModel.load_from_checkpoint(
    "/home/lorecampa/projects/jane_street_forecasting/experiments/saved/nn/model1/baseline_all.ckpt", 
    model=training_base_model,
    losses=[WeightedMSELoss()],
    loss_weights=[1]
)

inference_base_model = SimpleNNModel(79, hidden_dims=[256], dropout_rate=0.3, final_mult=5.0, use_tanh=True)
inference_model = JaneStreetBaseModel.load_from_checkpoint(
    "/home/lorecampa/projects/jane_street_forecasting/experiments/saved/nn/model1/baseline_all.ckpt", 
    model=inference_base_model,
    losses=[WeightedMSELoss()],
    loss_weights=[1]
)

# if torch.cuda.is_available():
#     training_model = training_model.cuda()
#     inference_model = inference_model.cuda()

optimizer = torch.optim.AdamW(
    training_model.parameters(), 
    lr=1e-6, 
    weight_decay=5e-4, 
    amsgrad=False, 
    betas=(0.9, 0.99))
training_model.train()
inference_model.eval()

/home/lorecampa/projects/jane_street_forecasting/.venv/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.0.post0, which is newer than your current Lightning version: v2.4.0


JaneStreetBaseModel(
  (model): SimpleNNModel(
    (model): Sequential(
      (0): BatchNorm1d(79, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.3, inplace=False)
      (2): Linear(in_features=79, out_features=256, bias=True)
      (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): SiLU()
      (5): Dropout(p=0.3, inplace=False)
      (6): Linear(in_features=256, out_features=1, bias=True)
      (7): Tanh()
    )
  )
  (losses): ModuleList(
    (0): WeightedMSELoss()
  )
)

In [11]:
for module in training_model.modules():
    # freeze batch normalization parameters
    if isinstance(module, nn.BatchNorm2d):
        if hasattr(module, 'weight'):
            module.weight.requires_grad_(False)
        if hasattr(module, 'bias'):
            module.bias.requires_grad_(False)
        module.eval() # do not change the running mean and var

In [12]:
def update_teacher(student_model, teacher_model, alpha=0.99):
    for teacher_param, student_param in zip(teacher_model.parameters(), student_model.parameters()):
        # exponential moving average
        teacher_param.data = alpha * teacher_param.data + (1.0 - alpha) * student_param.data

In [13]:
class JaneStreetBaseDataset(Dataset):
    
    def __init__(self, dataset: pl.DataFrame):
        super(JaneStreetBaseDataset, self).__init__()   
        self.dataset = dataset
        feature_cols = [f'feature_{i:02d}' for i in range(79)]
        self.X = torch.FloatTensor(self.dataset.select(feature_cols).to_numpy().astype(np.float32))
        self.y = torch.FloatTensor(self.dataset.select(['responder_6']).to_numpy().flatten().astype(np.float32))
        self.weights = torch.FloatTensor(self.dataset.select(['weight']).to_numpy().flatten().astype(np.float32))        
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):       
        return self.X[idx], self.y[idx], self.weights[idx]

In [14]:
FINE_TUNING = True
BATCH_SIZE = 32768

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    TRAIN_EVERY = 25 # start a new fine tuning every 14 days
    N_EPOCHS_PER_TRAIN = 2
    DATASET_MAX_SIZE = int(1e7)
else:
    TRAIN_EVERY = 2 # start a new fine tuning every 2 days
    N_EPOCHS_PER_TRAIN = 2
    DATASET_MAX_SIZE = int(1e6)
    
TIME_LIMIT = 30
FEATURE_COLS = [f'feature_{i:02d}' for i in range(79)]
MAX_FINE_TUNING_TIME_LIMIT = time.time() + 60 * 60 * 8 # after 8 hours, stop all the online learning
CLIP_NORM_VALUE = 5
dataset : pl.DataFrame | None = None
current_day_data : pl.DataFrame | None = None
date_idx = 0
start_train = False
train_dataloader = None
train_iterator = None
num_epochs_done = 0

COLUMNS = FEATURE_COLS + ['date_id', 'time_id', 'symbol_id', 'weight', 'responder_6']
dataset = pl.scan_parquet('/home/lorecampa/projects/jane_street_forecasting/dataset/train.parquet/partition_id=8/part-0.parquet')
dataset = dataset.select(COLUMNS) \
    .sort(['date_id', 'time_id', 'symbol_id']) \
    .with_columns(pl.col(FEATURE_COLS).fill_null(strategy='zero')) \
    .collect()[-DATASET_MAX_SIZE:, :]


def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    global FINE_TUNING, BATCH_SIZE, TRAIN_EVERY, DATASET_MAX_SIZE, N_EPOCHS_PER_TRAIN, TIME_LIMIT
    global date_idx, dataset, current_day_data, start_train, train_dataloader, num_epochs_done, train_iterator

    initial_time = time.time()
    FINE_TUNING = initial_time < MAX_FINE_TUNING_TIME_LIMIT
    if FINE_TUNING:
        if lags is not None:
            print('New day')
            lags_ = lags.select(
                pl.col('date_id').sub(1),
                pl.col(['time_id', 'symbol_id']),
                pl.col('responder_6_lag_1').alias('responder_6'),
            )
            if current_day_data is not None:
                current_day_data = current_day_data.join(lags_, on=['date_id', 'time_id', 'symbol_id'], 
                                                         how='left').fill_nan(0).fill_null(0)
                current_day_data = current_day_data.select(COLUMNS)
                dataset = dataset.vstack(current_day_data)
                dataset = dataset[-DATASET_MAX_SIZE:]
            # initialize current daily data
            current_day_data = test
            if not start_train:
                start_train = (date_idx + 1) % TRAIN_EVERY == 0
                if start_train:
                    print('Starting new fine tuning')
                    train_dataloader = JaneStreetBaseDataset(dataset)
                    train_dataloader = DataLoader(train_dataloader, shuffle=True, batch_size=BATCH_SIZE, num_workers=0)
                    train_iterator = iter(train_dataloader)
                    num_epochs_done = 0
            date_idx += 1
        else:
            # still the same date_id, no lags provided -> no data to add to replay buffer
            current_day_data = current_day_data.vstack(test)

        while start_train and time.time() - initial_time < TIME_LIMIT: 
            try:
                batch = next(train_iterator)
            except StopIteration:
                if num_epochs_done == N_EPOCHS_PER_TRAIN:
                    print(f'Completed Fine Tuning at time {test.select(pl.col("time_id").first()).item()}')
                    start_train = False
                    for training_param, inference_param in zip(training_model.parameters(), inference_model.parameters()):
                        inference_param.data = training_param.data
                    inference_model.eval()
                    break
                else:
                    num_epochs_done += 1
                    train_iterator = iter(train_dataloader)
                    batch = next(train_iterator)

            x, y, w = batch
            optimizer.zero_grad()
            y_out = training_model.forward(x.to(training_model.device)).squeeze()
            loss = training_model._compute_loss(y_out, y.to(training_model.device), w.to(training_model.device))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(training_model.parameters(), CLIP_NORM_VALUE)
            optimizer.step()

    if test.select(pl.col('is_scored').cast(pl.Int8).first()).item() > 0:
        predict_df = test.select([f'feature_{i:02d}' for i in range(79)]).fill_null(0).fill_nan(0)
        x = torch.tensor(predict_df.to_numpy(), dtype=torch.float32).to(inference_model.device)
        with torch.no_grad():
            preds = inference_model(x).cpu().numpy().flatten()
    else:
        preds = np.zeros(test.shape[0])
    predictions = test.select('row_id', pl.Series(preds).alias('responder_6'))

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    assert len(predictions) == len(test)

    return predictions

In [29]:
test_ds = pl.scan_parquet('/home/lorecampa/projects/jane_street_forecasting/dataset/train.parquet/partition_id=9/part-0.parquet').collect().with_row_index('row_id')
test_ds

row_id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,…,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
u32,i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1530,0,0,3.084694,1.153571,1.563784,0.697396,0.756759,2.580965,0.171311,1.126353,0.536153,0.05715,11,7,76,-0.656288,2.110188,0.145784,null,-0.203291,null,-1.238222,-2.294707,-0.06356,-0.148218,1.721362,0.64558,1.477857,0.528492,1.153077,0.466157,0.145568,-0.546845,-0.694435,-0.163897,…,0.680807,null,null,-2.786826,null,-1.2279,0.044606,null,-2.540213,-2.19028,0.385893,-0.460265,-0.415684,-0.45772,-1.333965,-2.23413,-0.352034,3.125156,0.493488,-0.9591,1.284456,-0.275493,null,null,4.188457,3.666236,0.848177,0.999516,0.417462,0.323897,0.601499,2.074103,0.746552,0.552013,3.071231,0.914794,0.997124
1,1530,0,1,2.232906,0.553354,1.730064,0.990195,0.61149,2.023031,0.319015,1.183371,0.562853,0.057789,11,7,76,-1.063518,1.037634,-0.255358,null,-0.318528,null,-1.46613,-2.160217,0.009386,0.042186,0.319811,0.14307,1.866907,1.238242,-1.986826,-0.476918,0.408439,-0.689795,-0.619278,0.081413,…,-0.00268,null,null,-1.736226,null,-2.354893,1.309985,null,-2.429267,-1.26697,0.385893,-0.24877,-0.286104,-0.455154,-1.797363,-2.535985,-0.734866,1.533782,0.033801,-0.960126,0.306505,-0.522036,null,null,1.138142,1.579439,0.179564,0.160609,-0.318671,-0.399384,-0.635306,2.092151,0.342582,0.757289,1.979042,0.967537,1.219739
2,1530,0,2,2.404948,1.532503,2.095852,0.919688,0.583715,2.330047,0.337096,1.262236,0.49605,0.073556,81,2,59,-1.001967,1.10577,-0.304426,null,-0.531873,null,-1.301579,-1.615271,0.454406,-0.188808,0.01512,-0.159487,1.379064,0.604568,0.736194,0.522007,-0.183058,-0.632819,-0.839542,-0.20955,…,0.793936,null,null,-1.191118,null,-2.190607,1.381697,null,-1.829545,-0.867858,0.385893,-0.295958,-0.386221,-0.345102,-1.598371,-2.111468,-0.780465,0.848857,-0.152994,-1.219395,0.359229,-0.636138,null,null,0.445388,0.300118,-0.043114,-0.065761,0.200878,-0.006571,0.51887,-0.344441,0.641694,-0.64604,-0.50626,0.739797,-2.041514
3,1530,0,3,1.986533,0.647099,1.68746,0.569406,1.061679,2.444131,0.150487,0.896543,0.705652,0.072545,4,3,11,-0.966606,0.415421,-0.528979,null,0.183964,null,-1.917427,-1.929856,-0.50175,-0.102371,-0.297677,-1.049185,0.933516,-0.60654,-0.039158,-0.27196,-0.920819,-0.926502,-0.783003,-0.13572,…,2.07314,null,null,0.205406,null,-0.760486,2.06341,null,4.528522,1.582996,0.385893,-0.396047,-0.146214,-0.228367,-2.062314,-1.421811,-0.851179,0.404792,-0.509694,-1.398495,0.231285,-0.595247,null,null,6.995454,7.133929,0.352965,0.526284,-0.349773,-0.235901,-0.428956,-1.903627,-1.214619,-0.4695,-2.590589,-0.946317,-0.390001
4,1530,0,4,2.742601,1.096778,1.551411,0.632113,0.368218,2.181873,0.214604,1.19423,0.261056,0.045426,15,1,9,-0.895004,0.047908,-0.683484,null,-0.638341,null,-1.55811,-1.203378,-0.305831,0.251725,0.222749,-0.902276,4.226925,1.233024,-2.274672,-1.065931,-0.229211,-0.597557,-0.990242,0.173607,…,0.549144,null,null,0.383838,null,-1.490257,2.019752,null,1.872912,0.571886,0.385893,-0.445097,-0.347954,-0.470278,-1.780053,-2.290865,-0.944827,0.027896,-0.5

In [17]:
from sklearn.metrics import r2_score


X_test = test_ds.select([f'feature_{i:02d}' for i in range(79)]).fill_null(strategy='zero').to_numpy()
y_test = test_ds['responder_6'].to_numpy().flatten()
w_test = test_ds['weight'].to_numpy().flatten()
with torch.no_grad():
    y_test_hat = inference_model(torch.tensor(X_test, dtype=torch.float32).to(inference_model.device)).cpu().numpy().flatten()
    
r2_score(y_true=y_test, y_pred=y_test_hat, sample_weight=w_test)

0.008284926414489746

In [18]:
import gc


del X_test, y_test, w_test, y_test_hat
gc.collect()

100

In [30]:
from tqdm import tqdm

def online_iterator(df: pl.DataFrame):
    assert df.select('date_id').n_unique() > 1, 'Dataset must contain at least 2 days'
    
    df_date_time_id = df.select('date_id', 'time_id').unique().sort('date_id', 'time_id').with_row_index('date_time_id')
    df = df.join(df_date_time_id, on=['date_id', 'time_id'], how='left', maintain_order='left')
    
    max_date_time_id = df_date_time_id['date_time_id'].max()
    min_date_id = df.select('date_id').min().item()
    
    responders = [f'responder_{i}' for i in range(9)]
    
    curr_idx:int = df_date_time_id.filter(pl.col('date_id').eq(min_date_id + 1))['date_time_id'].min()
    old_day = min_date_id

    with tqdm(total=max_date_time_id - curr_idx + 1) as pbar:
        while curr_idx <= max_date_time_id:
            curr_day = df_date_time_id[curr_idx]['date_id'].item()
            is_new_day = curr_day != old_day
            lags = None
            if is_new_day:
                print('new day')
                lags = df.filter(pl.col('date_id').eq(old_day)).select(pl.col('date_id').add(1), 'time_id', 'symbol_id', *[pl.col(r).alias(f'{r}_lag_1') for r in responders])
            
            old_day = curr_day

            batch = df.filter(pl.col('date_time_id').eq(curr_idx)).with_columns(pl.lit(True).alias('is_scored')).drop('date_time_id')
            
            yield batch, lags if lags is not None else None
            
            curr_idx += 1
            pbar.update(1)


In [31]:
for test_batch, lags in online_iterator(test_ds):
    break

  0%|          | 0/162624 [00:00<?, ?it/s]

new day


In [32]:
predict(test_batch, lags)

New day
Completed Fine Tuning at time 0


row_id,responder_6
u32,f32
35816,0.127188
35817,0.062084
35818,-0.043785
35819,-0.090471
35820,-0.128343
…,…
35850,0.003515
35851,0.097004
35852,0.055536
